# Decode Frame

Use this notebook to dynamically decode your frames. Just add a received frame into the prompt box bellow.

Test example (might not work if test IDs have changed): aa010ccdab0000adde0000efbe0000092455

In [4]:
from ipywidgets import Text, VBox, Button, Output
from IPython.display import display

# Create the input widget
data_input = Text(
    value='',
    placeholder='Enter the frame string in hexadecimal format',
    description='Frame:',
    disabled=False
)

# Create a button to process the input
process_button = Button(description="Decode Frame")

# Create an output widget to display results
output = Output()

# Display the widgets at the top of the notebook
display(VBox([data_input, process_button, output]))


In [5]:
from radcontrol.frame.frame_decoder import PacketFrame

def process_message(message):
    header = message[0:1]
    frame_id = message[1:2]
    payload_length = message[2]
    payload = message[3 : 3 + payload_length]
    crc_bytes = message[3 + payload_length : 3 + payload_length + 2]
    tail = message[-1:]

    packet = PacketFrame(header, frame_id, payload_length, payload, crc_bytes, tail)

    return packet

def display_packet_info(packet):
    # Display header information
    print("Packet Information:")
    print(f"  Header: {packet.header.hex()}")
    print(f"  Frame ID: {packet.frame_id.hex()}")
    print(f"  Payload Length: {packet.payload_length}")
    print(f"  Payload: {packet.payload.hex()}")
    print(f"  CRC Bytes: {packet.crc_bytes.hex()}")
    print(f"  Tail: {packet.tail.hex()}")

    # Check and display CRC status
    crc_valid = packet.check_crc()
    print(f"\nCRC Check: {'Valid' if crc_valid else 'Invalid'}")

    # Log message or decoded details
    log_message = packet.get_log_message("decoded")
    print(f"Payload: {log_message}")

# Test with your message
# message = bytes.fromhex("aa010ccdab0000adde0000efbe0000092455")

# Display formatted output

In [6]:
# Define the processing function
def process_data(b):
    with output:
        output.clear_output()
        data_string = data_input.value.strip()
        try:
            # Convert to bytes
            message = bytes.fromhex(data_string)
            # Process the message
            packet = process_message(message)
            # packet = process_message(message)
            display_packet_info(packet)
            # print(f"Processed Packet: {packet}")
        except ValueError:
            print("Invalid hexadecimal input.")
        except Exception as e:
            print(f"An error occurred: {e}")

# Link the button to the processing function
process_button.on_click(process_data)